In [1]:
# Carregar llibreries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np
import random
import time
from google.colab import files

In [2]:
# Carrega el fitxer Excel

# INFORME GPG: Inventari de drets immaterials - Dates de vigència

# nom_fitxer = '/content/Inventaridedretsimmaterials-Datesdevigència (2).xlsx'  # Canvia a la ruta correcta del teu fitxer
# df = pd.read_excel(nom_fitxer, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Carrega l'arxiu XLSX
uploaded = files.upload()

# Obtenir el nom de l'arxiu carregat
nom_arxiu = list(uploaded.keys())[0]

# Llegir l'arxiu XLSX amb pandas
df = pd.read_excel(nom_arxiu, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Selecciona les columnes que necessites
columnes_seleccionades = [
    "Codi registre",
    "Registre d'inscripció",
    "Data fi vigència",
    "Data inici vigència",
    "Data renovació",
    "Catàleg",
    "Estat",
    "Titular",
    "Número d'inventari"
]

df = df[columnes_seleccionades]


Saving Inventaridedretsimmaterials-Datesdevigència (11).xlsx to Inventaridedretsimmaterials-Datesdevigència (11).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Filtrar XLSX

# Obtenir la data d'avui
avui = datetime.now()

df2 = df[
    (df['Estat'] == 'Alta') &
    (df["Catàleg"] == 'Marca') &
    (df["Registre d'inscripció"] == "Oficina Espanyola de Patents i Marques (OEPM)") &
    (df["Data fi vigència"] < avui)
       ]
# print(df2["Codi registre"])

In [4]:
def separar(cadena):
    # Patró de cerca per la data (dd/mm/yyyy)
    patro_data = re.compile(r'\b(\d{1,2}/\d{1,2}/\d{4})\b')

    # Cerca la data a la cadena
    coincidencia_data = patro_data.search(cadena)

    if coincidencia_data:
        # Si es troba la data, separa el text i la data
        text = cadena[:coincidencia_data.start()].strip()
        data = coincidencia_data.group()
    else:
        # Si no es troba la data, retorna la cadena completa com a text i deixa la data buida
        text = cadena
        data = ""

    return text, data

# Exemple d'ús
# cadena = "PUBL.CONCES.RENOV. DE F.RESOL. 06/11/2023"
# text, data = separar(cadena)


In [5]:
def dades(codi):

    url = "http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=M" + codi

    dades = {}  # Crear un diccionari per emmagatzemar les dades

    dades['Codi'] = codi

    try:
        response = requests.get(url)

        if response.status_code == 200:

          try:

            # Parseja el contingut HTML amb BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Troba l'element <h4> amb el text "Actos de tramitación:"
            h4_actos = soup.find('h4', string=lambda text: "Actos de tramitación:" in text if text else False)
            h4_actos = h4_actos.find_next('div', class_='datos')

            # Troba tots els elements <td> amb l'atribut role="gridcell"
            td_elements = h4_actos.find_all('td', {'role': 'gridcell'})

            # Verifica si hi ha algun element
            if td_elements:
                # Obtenir l'últim element i extreure'n el text
                dades['last_td'] = td_elements[-1].get_text(strip=True)
                dades['Data publicació'] = td_elements[-2].get_text(strip=True)

                dades['Anotació'], dades['Data anotació'] = separar(dades['last_td'])

            # Busca l'element <h4> que contingui el text "Fecha presentación solicitud otorgada:"
            presentacio_h4 = soup.find('h4', string=lambda text: "Fecha presentación solicitud otorgada:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if presentacio_h4:
                dades['Data de presentació'] = presentacio_h4.find_next('p').get_text(strip=True)

            # Busca l'element <h4> que contingui el text "Estado:"
            estado_h4 = soup.find('h4', string=lambda text: "Estado:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if estado_h4:
                dades['Estat'] = estado_h4.find_next('p').get_text(strip=True)

            # Troba l'element <div> amb l'id "marco-imagen-sitamar"
            div_marco_imagen = soup.find('div', {'id': 'marco-imagen-sitamar'})

            if div_marco_imagen:
                # Troba l'element <img> dins del div
                img_element = div_marco_imagen.find('img')

                if img_element:
                    # Obté el valor de l'atribut src
                    src_valor = img_element.get('src', '')

                    # Imprimeix el resultat
                    dades['Imatge'] = "http://consultas2.oepm.es"+src_valor
                else:
                    dades['Imatge'] = ""

            # Troba l'element <h3>
            h3_element = soup.find('h3')

            if h3_element:
                # Obté el text de l'element <h3>
                text_h3 = h3_element.get_text(strip=True)

                # Utilitza una expressió regular per seleccionar el segon grup
                match = re.match(r'(.*)\s-\s(.*)', text_h3)

                if match:
                    segon_grup = match.group(2)
                    dades['Denominació'] = segon_grup

            # Troba l'element <h4>
            h4_element = soup.find('h4', string=lambda text: "Tipo:" in text if text else False)

            if h4_element:
                # Obtenir el text de l'element <h4>
                text_h4 = h4_element.find_next('p').get_text(strip=True)

                dades['Tipus'] = text_h4

            # Extreu el text que segueix a <span class="campo">Nombre:</span>
            nom_span = soup.find('span', class_='campo', string="Nombre:")
            if nom_span:
                text_nom = nom_span.find_next('div', class_='datos-sitamar-divSubRight').text.strip()
                # Comprova si hi ha l'etiqueta <br/>
                if "\n" in text_nom:
                    # Divideix el text en dos blocs amb l'etiqueta <br />
                    motiu, titular = map(str.strip, text_nom.split('\n', 1))
                    dades['Titular OEPM'] = titular
                    dades['Motiu'] = motiu
                else:
                    # Si no hi ha <br />, posa tot el text a 'titular'
                    dades['Titular OEPM'] = text_nom
                    dades['Motiu'] = ""

            dades['Url'] = url
            dades['Titular GPG'] = df2['Titular'][df2['Codi registre'] == codi].values[0]
            dades['Inici vigència GPG'] = pd.to_datetime(df2["Data inici vigència"][df2['Codi registre'] == codi]).dt.date.values[0]
            dades['Fi vigència GPG'] = pd.to_datetime(df2["Data fi vigència"][df2['Codi registre'] == codi]).dt.date.values[0]
            dades['Núm. secció'] = df2["Número d'inventari"][df2['Codi registre'] == codi].values[0]
          except:
              dades['Estat'] = "Núm. registre incorrecte"

        else:
            print(f"S'ha produït un error en la sol·licitud. Codi d'estat: {response.status_code}")

    except requests.RequestException as e:
        print(f"Error de connexió: {e}")

    return dades  # Retorna el diccionari amb les dades

In [6]:
'''
# Exemple d'ús

resultats = dades("2551345")

# Ara pots fer el que vulguis amb les dades obtingudes
if resultats:
    # Fer quelcom amb les dades, com ara imprimir-les
    print("Dades obtingudes:")
    print(f"codi: {resultats.get('Codi')}")
    print(f"denominació: {resultats.get('Denominació')}")
    print(f"estado_text: {resultats.get('Estat')}")
    print(f"data_publicacio: {resultats.get('Data publicació')}")
    print(f"anotacio: {resultats.get('Anotació')}")
    print(f"anotacio_data: {resultats.get('Data anotació')}")
    print(f"imatge: {resultats.get('Imatge')}")
    print(f"titular: {resultats.get('Titular OEPM')}")
    print(f"motiu: {resultats.get('Motiu')}")
    print(f"tipus: {resultats.get('Tipus')}")
    print(f"Url: {resultats.get('Url')}")
'''

'\n# Exemple d\'ús\n\nresultats = dades("2551345")\n\n# Ara pots fer el que vulguis amb les dades obtingudes\nif resultats:\n    # Fer quelcom amb les dades, com ara imprimir-les\n    print("Dades obtingudes:")\n    print(f"codi: {resultats.get(\'Codi\')}")\n    print(f"denominació: {resultats.get(\'Denominació\')}")\n    print(f"estado_text: {resultats.get(\'Estat\')}")\n    print(f"data_publicacio: {resultats.get(\'Data publicació\')}")\n    print(f"anotacio: {resultats.get(\'Anotació\')}")\n    print(f"anotacio_data: {resultats.get(\'Data anotació\')}")\n    print(f"imatge: {resultats.get(\'Imatge\')}")\n    print(f"titular: {resultats.get(\'Titular OEPM\')}")\n    print(f"motiu: {resultats.get(\'Motiu\')}")\n    print(f"tipus: {resultats.get(\'Tipus\')}")\n    print(f"Url: {resultats.get(\'Url\')}")\n'

In [7]:
# Baixar la informació

marques_revisades = pd.DataFrame()

id = df2["Codi registre"]

print("Marques a processar: ", len(id), "\n")

for index, x in enumerate(id):
    print(str(round(index/len(id)*100, 2))+"%")
    resultats = dades(str(x))
    marques_revisades = pd.concat([marques_revisades, pd.DataFrame([resultats])], ignore_index=True)
    time.sleep(random.uniform(10, 15))
print("100.00 %")

marques_revisades.drop('last_td', axis=1, inplace=True)
marques_revisades

Marques a processar:  31 

0.0%
3.23%
6.45%
9.68%
12.9%
16.13%
19.35%
22.58%
25.81%
29.03%
32.26%
35.48%
38.71%
41.94%
45.16%
48.39%
51.61%
54.84%
58.06%
61.29%
64.52%
67.74%
70.97%
74.19%
77.42%
80.65%
83.87%
87.1%
90.32%
93.55%
96.77%
100.00 %


,Codi,Data publicació,Anotació,Data anotació,Data de presentació,Estat,Imatge,Tipus,Titular OEPM,Motiu,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG,Núm. secció,Denominació
0,1806112,23/02/2024,ANOTACION SOLICITUD RENOVACION,,25/02/1994,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Gráfica,GENERALITAT DE CATALUNYA - DEPARTAMENT D'ENSEN...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Educació,1994-02-25,2024-02-25,15.0,NaN
1,2568170,11/12/2013,PUBL.CONCES.RENOV. DE F.RESOL.,04/12/2013,20/11/2003,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Mixta,GENERALITAT DE CATALUNYA-DEPARTAMENT D'ENSENYA...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Educació,2003-11-20,2023-11-20,21.0,CANAL EDU365
2,3111777,20/11/2023,ANOTACION SOLICITUD RENOVACION,,20/02/2014,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Mixta,SERVEI CATALA DE TRANSIT,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Servei Català de Trànsit,2014-02-20,2024-02-20,53.0,SIAVT
3,3111642,19/10/2023,ANOTACION SOLICITUD RENOVACION,,19/02/2014,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Interior,2014-02-19,2024-02-19,60.0,BOMBERS
4,3111646,19/10/2023,ANOTACION SOLICITUD RENOVACION,,19/02/2014,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Interior,2014-02-19,2024-02-19,61.0,GENERALITAT DE CATALUNYA BOMBERS
5,1752508,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,26/03/1993,En vigor,,Denominativa,GENERALITAT DE CATALUNYA,POR CAMBIO NOMBRE NUM. 2012.09876 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,1993-03-26,2023-03-26,2362.0,CATALUNYA BARRI A BARRI
6,1752509,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,26/03/1993,En vigor,,Denominativa,GENERALITAT DE CATALUNYA,POR CAMBIO NOMBRE NUM. 2012.09876 DE F. RESOL....,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,1993-03-26,2023-03-26,2363.0,CONVIURE
7,2570996,04/12/2023,ANOTACION SOLICITUD RENOVACION,,05/12/2003,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Denominativa,GENERALITAT DE CATALUNYA DEPARTAMENT DE LA PRE...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,2003-12-05,2023-12-05,2373.0,CLIC. FOTOPERIODISME JOVE
8,3101811,22/02/2024,ANOTACION SOLICITUD RENOVACION,,27/11/2013,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,2013-11-27,2023-11-27,2376.0,XARXA OMNIA
9,1776353,22/02/2013,PUBL.CONCES.RENOV. DE F.RESOL.,12/02/2013,21/07/1993,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,Mixta,INSTITUT DE DIAGNOSTIC PER LA IMATGE,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Institut de Diagnòstic per la Imatge (IDI),1993-07-21,2023-07-21,2623.0,IDI INSTITUT DE DIAGNOSTIC PER LA IMATGE


In [8]:
import pandas as pd

# Defineix l'ordre desitjat de les columnes
ordre_columnes = ['Codi', 'Núm. secció', 'Denominació', 'Tipus', 'Estat', 'Data de presentació', 'Data publicació', 'Anotació', 'Data anotació', 'Titular OEPM', 'Motiu', 'Imatge', 'Url', 'Titular GPG', 'Inici vigència GPG', 'Fi vigència GPG']

# Crea un DataFrame amb les columnes desitjades
marques_revisades_2 = pd.DataFrame(columns=ordre_columnes)

# Afegeix les dades existents al teu DataFrame original
marques_revisades_2 = marques_revisades_2.append(marques_revisades, ignore_index=True)

# Reordena les columnes del DataFrame
marques_revisades = marques_revisades_2[ordre_columnes]
marques_revisades = marques_revisades.fillna("")

marques_revisades

<ipython-input-8-6bd39ead835e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  marques_revisades_2 = marques_revisades_2.append(marques_revisades, ignore_index=True)


,Codi,Núm. secció,Denominació,Tipus,Estat,Data de presentació,Data publicació,Anotació,Data anotació,Titular OEPM,Motiu,Imatge,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG
0,1806112,15.0,,Gráfica,En vigor,25/02/1994,23/02/2024,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA - DEPARTAMENT D'ENSEN...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Educació,1994-02-25,2024-02-25
1,2568170,21.0,CANAL EDU365,Mixta,En vigor,20/11/2003,11/12/2013,PUBL.CONCES.RENOV. DE F.RESOL.,04/12/2013,GENERALITAT DE CATALUNYA-DEPARTAMENT D'ENSENYA...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Educació,2003-11-20,2023-11-20
2,3111777,53.0,SIAVT,Mixta,En vigor,20/02/2014,20/11/2023,ANOTACION SOLICITUD RENOVACION,,SERVEI CATALA DE TRANSIT,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Servei Català de Trànsit,2014-02-20,2024-02-20
3,3111642,60.0,BOMBERS,Mixta,En vigor,19/02/2014,19/10/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Interior,2014-02-19,2024-02-19
4,3111646,61.0,GENERALITAT DE CATALUNYA BOMBERS,Mixta,En vigor,19/02/2014,19/10/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Interior,2014-02-19,2024-02-19
5,1752508,2362.0,CATALUNYA BARRI A BARRI,Denominativa,En vigor,26/03/1993,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,GENERALITAT DE CATALUNYA,POR CAMBIO NOMBRE NUM. 2012.09876 DE F. RESOL....,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,1993-03-26,2023-03-26
6,1752509,2363.0,CONVIURE,Denominativa,En vigor,26/03/1993,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,GENERALITAT DE CATALUNYA,POR CAMBIO NOMBRE NUM. 2012.09876 DE F. RESOL....,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,1993-03-26,2023-03-26
7,2570996,2373.0,CLIC. FOTOPERIODISME JOVE,Denominativa,En vigor,05/12/2003,04/12/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA DEPARTAMENT DE LA PRE...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,2003-12-05,2023-12-05
8,3101811,2376.0,XARXA OMNIA,Mixta,En vigor,27/11/2013,22/02/2024,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,2013-11-27,2023-11-27
9,1776353,2623.0,IDI INSTITUT DE DIAGNOSTIC PER LA IMATGE,Mixta,En vigor,21/07/1993,22/02/2013,PUBL.CONCES.RENOV. DE F.RESOL.,12/02/2013,INSTITUT DE DIAGNOSTIC PER LA IMATGE,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Institut de Diagnòstic per la Imatge (IDI),1993-07-21,2023-07-21


In [9]:
# Crea columna ACCIÓ aplicant condicions

from datetime import timedelta

def process_row(row):
    date_format = "%d/%m/%Y"
    if row['Inici vigència GPG']:
      inici_vigencia = row['Inici vigència GPG'].strftime(date_format)

      if row['Estat'] == "Núm. registre incorrecte":
        return "Núm. registre incorrecte"
      elif row['Data de presentació'] != inici_vigencia:
          return "Dates no coincidents"
      elif row['Estat'] == "Caducado":
          return "Baixa"
      elif row['Data anotació'] == "":
          fi_vigencia = row['Fi vigència GPG']
          if fi_vigencia < (datetime.now().date() - timedelta(days=365)):
              return "Més d'un any caducada"
          else:
              return "Deixar pendent"
      elif datetime.strptime(row['Data anotació'], date_format).date() > row['Fi vigència GPG']:
          return "Renovat"
      elif row['Fi vigència GPG'] < (datetime.now().date() - timedelta(days=365)):
          return "Més d'un any caducada"
      else:
          return "Deixar pendent"

# Aplica la funció a cada fila del DataFrame
marques_revisades['Acció'] = marques_revisades.apply(process_row, axis=1)

# Mostra el DataFrame amb la nova columna
marques_revisades


,Codi,Núm. secció,Denominació,Tipus,Estat,Data de presentació,Data publicació,Anotació,Data anotació,Titular OEPM,Motiu,Imatge,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG,Acció
0,1806112,15.0,,Gráfica,En vigor,25/02/1994,23/02/2024,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA - DEPARTAMENT D'ENSEN...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Educació,1994-02-25,2024-02-25,Deixar pendent
1,2568170,21.0,CANAL EDU365,Mixta,En vigor,20/11/2003,11/12/2013,PUBL.CONCES.RENOV. DE F.RESOL.,04/12/2013,GENERALITAT DE CATALUNYA-DEPARTAMENT D'ENSENYA...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Educació,2003-11-20,2023-11-20,Deixar pendent
2,3111777,53.0,SIAVT,Mixta,En vigor,20/02/2014,20/11/2023,ANOTACION SOLICITUD RENOVACION,,SERVEI CATALA DE TRANSIT,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Servei Català de Trànsit,2014-02-20,2024-02-20,Deixar pendent
3,3111642,60.0,BOMBERS,Mixta,En vigor,19/02/2014,19/10/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Interior,2014-02-19,2024-02-19,Deixar pendent
4,3111646,61.0,GENERALITAT DE CATALUNYA BOMBERS,Mixta,En vigor,19/02/2014,19/10/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament d'Interior,2014-02-19,2024-02-19,Deixar pendent
5,1752508,2362.0,CATALUNYA BARRI A BARRI,Denominativa,En vigor,26/03/1993,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,GENERALITAT DE CATALUNYA,POR CAMBIO NOMBRE NUM. 2012.09876 DE F. RESOL....,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,1993-03-26,2023-03-26,Deixar pendent
6,1752509,2363.0,CONVIURE,Denominativa,En vigor,26/03/1993,29/04/2021,"PERSONACION AGENTE, AGENTE ANTERIOR: 0986 (POD...",,GENERALITAT DE CATALUNYA,POR CAMBIO NOMBRE NUM. 2012.09876 DE F. RESOL....,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,1993-03-26,2023-03-26,Deixar pendent
7,2570996,2373.0,CLIC. FOTOPERIODISME JOVE,Denominativa,En vigor,05/12/2003,04/12/2023,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA DEPARTAMENT DE LA PRE...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,2003-12-05,2023-12-05,Deixar pendent
8,3101811,2376.0,XARXA OMNIA,Mixta,En vigor,27/11/2013,22/02/2024,ANOTACION SOLICITUD RENOVACION,,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Departament de Drets Socials,2013-11-27,2023-11-27,Deixar pendent
9,1776353,2623.0,IDI INSTITUT DE DIAGNOSTIC PER LA IMATGE,Mixta,En vigor,21/07/1993,22/02/2013,PUBL.CONCES.RENOV. DE F.RESOL.,12/02/2013,INSTITUT DE DIAGNOSTIC PER LA IMATGE,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Institut de Diagnòstic per la Imatge (IDI),1993-07-21,2023-07-21,Deixar pendent


In [10]:
marques_revisades.to_excel(f'marques_oepm_revisades_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx', index=False)

In [11]:
files.download(f'marques_oepm_revisades_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>